In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from matplotlib import rc
from numpy import inf
import os
from os import listdir

rc('text', usetex=True)

import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

#
mpl.rcParams.update({'font.size': 19})
#mpl.rcParams.update({'legend.fontsize': 18})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

#import mplhep as hep
#hep.set_style(hep.style.ROOT)
print(pd.__version__)

0.25.3


In [2]:
#
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
# Define default plot styles
plot_style_0 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'linestyle': '--',
    'density': True
}

plot_style_1 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'density': True
}

plot_style_2 = {'alpha': 0.5, 'density': True}

plot_style_1A = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'density': False
}

plot_style_2A = {'alpha': 0.5, 'density': False}


In [4]:
def applyCut(inputDataframe, cut, text=None):
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.query(cut)
    if text:
        print (text, cutDataframe.shape[0], ' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut))
    return cutDataframe

In [5]:
def applyCutsJets(df,isMC=False):
    temp = df
    temp.eval('rec_mass = sqrt( (rec_Epip + rec_Epim)**2 -(rec_pipx + rec_pimx)**2 - (rec_pipy + rec_pimy)**2 - (rec_pipz + rec_pimz)**2) ', inplace=True)
    temp.eval('rec_ppt = sqrt( rec_px*rec_px + rec_py*rec_py)', inplace=True)
    temp.eval('rec_pp = sqrt( rec_px*rec_px + rec_py*rec_py + rec_pz*rec_pz)', inplace=True)
    temp.eval('rec_pipp = sqrt( rec_pipx*rec_pipx + rec_pipy*rec_pipy + rec_pipz*rec_pipz)', inplace=True)
    temp.eval('rec_pimp = sqrt( rec_pimx*rec_pimx + rec_pimy*rec_pimy + rec_pimz*rec_pimz)', inplace=True)

    temp.eval('rec_t2  = rec_ppt*rec_ppt',inplace=True)
    temp.eval('rec_ptheta = 180.0*arctan(rec_py/rec_pz)/3.14159',inplace=True)
    temp.eval('rec_piptheta = 180.0*arctan(rec_pipy/rec_pipz)/3.14159',inplace=True)
    temp.eval('rec_pimtheta = 180.0*arctan(rec_pimy/rec_pimz)/3.14159',inplace=True)

    
    temp.eval('gen_mass = sqrt( (gen_Epip + gen_Epim)**2 -(gen_pipx + gen_pimx)**2 - (gen_pipy + gen_pimy)**2 - (gen_pipz + gen_pimz)**2) ', inplace=True)
    temp.eval('gen_ppt = sqrt( gen_px*gen_px + gen_py*gen_py)', inplace=True)
    temp.eval('gen_pp = sqrt( gen_px*gen_px + gen_py*gen_py + gen_pz*gen_pz)', inplace=True)
    temp.eval('gen_t2  = gen_ppt*gen_ppt',inplace=True)
    temp.eval('gen_ptheta = 180.0*arctan(gen_py/gen_pz)/3.14159',inplace=True)
    temp.eval('gen_piptheta = 180.0*arctan(gen_pipy/gen_pipz)/3.14159',inplace=True)
    temp.eval('gen_pimtheta = 180.0*arctan(gen_pimy/gen_pimz)/3.14159',inplace=True)
    
    
    
    temp = applyCut(temp, 'pass_reco==0 | rec_pp > 0.32', 'proton p > 0.320 GeV')
    temp = applyCut(temp, 'pass_reco==0 | rec_ptheta > 10.0', 'proton theta > 10 degrees')
    temp = applyCut(temp, 'gen_ptheta > 10.0', 'generated proton p > 10 degrees')
    temp = applyCut(temp, 'gen_pz > .50', 'generated proton pz > 0.5 GeV')

    temp = applyCut(temp, 'pass_reco==0 | rec_ptheta < 120.0', 'pi+ theta < 120')

    temp = applyCut(temp, 'pass_reco==0 | rec_pipz !=0.0', 'pipz!=0')
    temp = applyCut(temp, 'pass_reco==0 | rec_pimz !=0.0', 'pimz!=0')
    temp = applyCut(temp, 'pass_reco==0 | rec_pz !=0.0', 'pz!=0')

    return temp

In [33]:
%%time
df_gen = np.load('gen_ps_2',allow_pickle=True)
df_rec = np.load('rec_ps_2',allow_pickle=True)
#joineddf = df_gen.join(df_rec,  rsuffix='_rec', lsuffix='_gen')

#altnpz = np.load('ppip_real.npz')
#altmc= pd.DataFrame.from_dict({item: altnpz[item] for item in altnpz.keys()})


CPU times: user 4.22 ms, sys: 77 ms, total: 81.3 ms
Wall time: 80.4 ms


In [34]:
df_gen.head()

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
mc.hist(figsize=(30,30))
plt.show()

In [ ]:
altmc.hist(figsize=(30,30),bins=100)
plt.show()

## Get dataframes, apply selection

In [ ]:
mc_tag = 'phasespace' ##could be "sys_0 or sys_1, etc"
altmc_tag  = 'realistic'
period = 'photoproduction'

## Define the pass_reco and pass_tuth flags. i.e. whether row in the dataframe jet has a reco-level jet, and whether it has a valid truth-level jet. 

In [ ]:
%%time 
mc['pass_reco'] = np.where(mc['rec_Ep']>0, 1, 0)
mc['pass_truth'] = np.where(mc['gen_Ep']>0, 1, 0)
altmc['pass_reco'] = np.where(altmc['rec_Ep']>0, 1, 0)
altmc['pass_truth'] = np.where(altmc['gen_Ep']>0, 1, 0)


In [ ]:
%%time 

print('Selecting data events\n')
mc = applyCutsJets(mc)
altmc = applyCutsJets(altmc)

In [ ]:
mc.hist(figsize=(30,30),bins=100)
plt.show()

In [ ]:
plt.hist(mc.query('pass_reco==1')['rec_ptheta'], bins=100,**plot_style_1A,label='reco')
plt.hist(mc.query('pass_reco==1')['gen_ptheta'], bins=100,**plot_style_2A,label='gen|reco')
plt.hist(mc['gen_ptheta'], bins=100,**plot_style_2A,label='gen|reco')

#plt.hist(altmc.query('pass_reco==1')['rec_ppt'], range=(0,2),bins=100,**plot_style_1,label='reco')
#plt.hist(altmc.query('pass_reco==1')['gen_ppt'], range=(0,2),bins=100,**plot_style_2,label='gen | reco')
plt.xlabel('p theta')
plt.legend()
plt.show()

In [ ]:
mc.query('pass_reco==1').hist(figsize=(30,30),bins=100)
altmc.query('pass_reco==1').hist(figsize=(30,30),bins=100,color='red')

plt.show()

In [ ]:
mc.query('pass_reco==0').hist(figsize=(30,30),bins=100)
altmc.query('pass_reco==0').hist(figsize=(30,30),bins=100,color='red')

plt.show()

In [ ]:
plt.hist(mc.query('pass_reco==1')['rec_pp'], range=(0,6),bins=100,**plot_style_1,label='reco')
plt.hist(mc.query('pass_reco==1')['gen_pp'], range=(0,6),bins=100,**plot_style_2,label='gen ')

plt.hist(altmc.query('pass_reco==1')['rec_pp'], range=(0,6),bins=100,**plot_style_1,label='reco')
plt.hist(altmc.query('pass_reco==1')['gen_pp'], range=(0,6),bins=100,**plot_style_2,label='gen')
#plt.yscale('log')
plt.legend()
plt.xlabel('proton momentum [GeV]')
plt.show()

In [ ]:
plt.hist(mc.query('pass_reco==1')['rec_mass'], range=(0,2),bins=100,**plot_style_1,label='MC1 reco')
plt.hist(mc.query('pass_reco==1')['gen_mass'], range=(0,2),bins=100,**plot_style_2,label='MC1 gen |reco ')

plt.hist(altmc.query('pass_reco==1')['rec_mass'], range=(0,2),bins=100,**plot_style_1,label='MC2 reco ')
plt.hist(altmc.query('pass_reco==1')['gen_mass'], range=(0,2),bins=100,**plot_style_2,label='MC2 gen | reco')

plt.xlabel('Dipion invariant mass [GeV]')
plt.legend()
plt.show()

In [ ]:
plt.hist(mc.query('pass_reco==1')['rec_ptheta'], range=(0,180),bins=100,**plot_style_1,label='MC1 reco')
plt.hist(mc.query('pass_reco==1')['gen_ptheta'], range=(0,180),bins=100,**plot_style_2,label='MC1 gen |reco ')

plt.hist(altmc.query('pass_reco==1')['rec_ptheta'], range=(0,180),bins=100,**plot_style_1,label='MC2 reco ')
plt.hist(altmc.query('pass_reco==1')['gen_ptheta'], range=(0,180),bins=100,**plot_style_2,label='MC2 gen | reco')

plt.xlabel('proton angle ')
plt.legend()
plt.show()

In [ ]:
plt.hist(mc.query('pass_reco==1')['rec_piptheta'], range=(0,180),bins=100,**plot_style_1,label='MC1 reco')
plt.hist(mc.query('pass_reco==1')['gen_piptheta'], range=(0,180),bins=100,**plot_style_2,label='MC1 gen |reco ')

plt.hist(altmc.query('pass_reco==1')['rec_piptheta'], range=(0,180),bins=100,**plot_style_1,label='MC2 reco ')
plt.hist(altmc.query('pass_reco==1')['gen_piptheta'], range=(0,180),bins=100,**plot_style_2,label='MC2 gen | reco')

plt.xlabel('pip theta')
plt.legend()
plt.show()

### Define UniFold Algorithm


In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Patch
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
# Input multiple observables as tuple or list (i.e. theta0_G = (obs_1, obs_2, ...))


def weighted_binary_crossentropy(y_true, y_pred):
    weights = tf.gather(y_true, [1], axis=1) # event weights
    y_true = tf.gather(y_true, [0], axis=1) # actual y_true for loss
    
    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = -weights * ((y_true) * K.log(y_pred) +
                         (1 - y_true) * K.log(1 - y_pred))
    
    return K.mean(t_loss)
def multifold(num_observables, iterations, theta0_G, theta0_S,
              theta_unknown_S,weights_MC_sim=None,weights_MC_data=None,verbose=1):
    
    if weights_MC_sim is None:
        weights_MC_sim = np.ones(len(theta0_S))
    
    if weights_MC_data is None:
        weights_MC_data = np.ones(len(theta_unknown_S))
    
    
    theta0 = np.stack([theta0_G, theta0_S], axis=1)
    labels0 = np.zeros(len(theta0))
    theta_unknown = np.stack([theta_unknown_S, theta_unknown_S], axis=1)
    labels1 = np.ones(len(theta0_G))
    labels_unknown = np.ones(len(theta_unknown_S))
    
    xvals_1 = np.concatenate((theta0_S, theta_unknown_S))
    yvals_1 = np.concatenate((labels0, labels_unknown))
    xvals_2 = np.concatenate((theta0_G, theta0_G))
    yvals_2 = np.concatenate((labels0, labels1))
    weights = np.empty(shape=(iterations, 2, len(theta0_G)))
    
    inputs = Input((num_observables, ))
    hidden_layer_1 = Dense(50, activation='relu')(inputs)
    dropoutlayer = Dropout(0.2)(hidden_layer_1)
    hidden_layer_2 = Dense(50, activation='relu')(dropoutlayer)
    hidden_layer_3 = Dense(50, activation='relu')(hidden_layer_2)
    outputs = Dense(1, activation='sigmoid')(hidden_layer_3)
    model = Model(inputs=inputs, outputs=outputs)
    earlystopping = EarlyStopping(patience=10,
                              verbose=verbose,
                              restore_best_weights=True)
    
    # from NN (DCTR) 
    def reweight(events):
        f = model.predict(events, batch_size=1000)
        weights = f / (1. - f)
        return np.squeeze(np.nan_to_num(weights))
    
    
    
    weights_pull = weights_MC_sim
    weights_push = weights_MC_sim
    
    #weights_pull = np.ones(len(theta0_S))
    #weights_push = np.ones(len(theta0_S))
    
    history = {}
    history['step1'] = []
    history['step2'] = []
    for i in range(iterations):
        print("\nITERATION: {}\n".format(i + 1))
        print("STEP 1\n")
        
        weights_1 = np.concatenate((weights_push,weights_MC_data ))
        
        
      
        X_train_1, X_test_1, Y_train_1, Y_test_1, w_train_1, w_test_1 = train_test_split(
        xvals_1, yvals_1, weights_1)
        
        Y_train_1 = np.stack((Y_train_1, w_train_1), axis=1)
        Y_test_1 = np.stack((Y_test_1, w_test_1), axis=1)
            
            
        batch_size=1000
        model.compile(loss=weighted_binary_crossentropy,
                  optimizer='Adam',
                  metrics=['accuracy'])
        
        hist_s1 =  model.fit(X_train_1,
              Y_train_1,
              epochs=200,
              batch_size=batch_size,
              validation_data=(X_test_1, Y_test_1),
              callbacks=[earlystopping],
              verbose=verbose)
        
        history['step1'].append(hist_s1)
        weights_pull = weights_push * reweight(theta0_S)
        weights_pull[theta0_S[:,0]==-10] = 1
        weights[i, :1, :] = weights_pull
        
        print("\nSTEP 2\n")
        weights_2 = np.concatenate((weights_MC_sim, weights_pull))

        #weights_2 = np.concatenate((np.ones(len(theta0_G)), weights_pull))
        # ones for MC Truth (not MC weights), actual weights for (reweighted) MC Truth
        X_train_2, X_test_2, Y_train_2, Y_test_2, w_train_2, w_test_2 = train_test_split(
            xvals_2, yvals_2, weights_2)
    
        # zip ("hide") the weights with the labels
        Y_train_2 = np.stack((Y_train_2, w_train_2), axis=1)
        Y_test_2 = np.stack((Y_test_2, w_test_2), axis=1)
        model.compile(loss=weighted_binary_crossentropy,
                  optimizer='Adam',
                  metrics=['accuracy'])
        hist_s2 = model.fit(X_train_2,
              Y_train_2,
              epochs=200,
              batch_size=batch_size,
              validation_data=(X_test_2, Y_test_2),
              callbacks=[earlystopping],
              verbose=verbose)
        
        history['step2'].append(hist_s2)

        #weights_push = reweight(theta0_G)
        weights_push = weights_MC_sim * reweight(theta0_G)

        weights[i, 1:2, :] = weights_push
   
    return weights, model, history

    





In [ ]:
print(tf.__version__)
print(tensorflow.keras.__version__)

print(np.__version__)

In [ ]:
tf.random.set_seed(1234)
np.random.seed(1234)

## Define features with data

In [ ]:
data = altmc.query('pass_reco==1')#[:300000]
n = int(len(data)*30)
mc_cut = mc[:n]

print(len(mc_cut))
print(len(data))

In [ ]:
print(len(data))
print(len(mc_cut.query('pass_reco==1'))) ##only what is really reconstructed counts
print(len(mc_cut)) ##only what is really reconstructed counts
ratio = len(mc_cut.query('pass_reco==1'))/len(data.query('pass_reco==1'))
print(ratio)

In [ ]:
mc_cut.keys()

In [ ]:
px_0_G       = mc_cut['gen_px']
px_0_S       = mc_cut['rec_px']
px_unknown_G = data['gen_px']
px_unknown_S = data['rec_px']

py_0_G       = mc_cut['gen_py']
py_0_S       = mc_cut['rec_py']
py_unknown_G = data['gen_py']
py_unknown_S = data['rec_py']

pz_0_G       = mc_cut['gen_pz']
pz_0_S       = mc_cut['rec_pz']
pz_unknown_G = data['gen_pz']
pz_unknown_S = data['rec_pz']



pipx_0_G       = mc_cut['gen_pipx']
pipx_0_S       = mc_cut['rec_pipx']
pipx_unknown_G  = data['gen_pipx']
pipx_unknown_S = data['rec_pipx']

pipy_0_G       = mc_cut['gen_pipy']
pipy_0_S       = mc_cut['rec_pipy']
pipy_unknown_G  = data['gen_pipy']
pipy_unknown_S = data['rec_pipy']

pipz_0_G       = mc_cut['gen_pipz']
pipz_0_S       = mc_cut['rec_pipz']
pipz_unknown_G  = data['gen_pipz']
pipz_unknown_S = data['rec_pipz']


pimx_0_G       = mc_cut['gen_pimx']
pimx_0_S       = mc_cut['rec_pimx']
pimx_unknown_G  = data['gen_pimx']
pimx_unknown_S = data['rec_pimx']

pimy_0_G       = mc_cut['gen_pimy']
pimy_0_S       = mc_cut['rec_pimy']
pimy_unknown_G  = data['gen_pimy']
pimy_unknown_S = data['rec_pimy']

pimz_0_G       = mc_cut['gen_pimz']
pimz_0_S       = mc_cut['rec_pimz']
pimz_unknown_G  = data['gen_pimz']
pimz_unknown_S = data['rec_pimz']

mass_0_G           = mc_cut['gen_mass']
mass_0_S           = mc_cut['rec_mass']
mass_unknown_G     = data['gen_mass']
mass_unknown_S     = data['rec_mass']

In [ ]:
theta0_G=np.c_[(px_0_G, py_0_G, pz_0_G,
                pipx_0_G, pipy_0_G, pipz_0_G,
                pimx_0_G, pimy_0_G,  pimz_0_G)]
theta0_S=np.c_[(px_0_S, py_0_S, pz_0_S,
                pipx_0_S, pipy_0_S, pipz_0_S, 
                pimx_0_S, pimy_0_S,  pimz_0_S)]
theta_unknown_S =np.c_[(px_unknown_S, py_unknown_S, pz_unknown_S,
                       pipx_unknown_S, pipy_unknown_S, pipz_unknown_S,
                       pimx_unknown_S, pimy_unknown_S,  pimz_unknown_S)]

theta_unknown_G =np.c_[(px_unknown_G, py_unknown_G, pz_unknown_G,
                       pipx_unknown_G, pipy_unknown_G, pipz_unknown_G,
                        pimx_unknown_G, pimy_unknown_G,  pimz_unknown_G)]

In [ ]:
theta0_G=np.c_[(px_0_G, py_0_G, pz_0_G,
                pipx_0_G, pipy_0_G, pipz_0_G
              )]
theta0_S=np.c_[(px_0_S, py_0_S, pz_0_S,
                pipx_0_S, pipy_0_S, pipz_0_S 
                )]
theta_unknown_S =np.c_[(px_unknown_S, py_unknown_S, pz_unknown_S,
                       pipx_unknown_S, pipy_unknown_S, pipz_unknown_S
                       )]

theta_unknown_G =np.c_[(px_unknown_G, py_unknown_G, pz_unknown_G,
                       pipx_unknown_G, pipy_unknown_G, pipz_unknown_G
                      )]

In [ ]:
print(len(mc_cut.query('pass_reco==1')))
print(len(mc_cut.query('pass_truth==0')))
print(len(mc_cut))

In [ ]:
pass_reco = np.array(mc_cut['pass_reco'])
pass_truth = np.array(mc_cut['pass_truth'])
theta0_S[:,0][pass_reco==0] = -10
theta0_G[:,0][pass_truth==0] = -10



In [ ]:
bins = np.linspace(-3,3, 101)

fig, ax = plt.subplots(1,
                       2,
                       figsize=(12, 5),
                       constrained_layout=True,
                       sharey=True)

ax[0].set_xlabel('$x_0$')
ax[0].set_ylabel('Events per bin (normalized)')
ax[0].hist(theta0_G[theta0_G[:,0]!=-10][:,0], bins=bins, **plot_style_2, label='Generation')
ax[0].hist(theta0_S[theta0_S[:,0]!=-10][:,0], bins=bins, **plot_style_2, label='Simulation')
legend = ax[0].legend(
    loc='best',
    frameon=True)
plt.setp(legend.get_title(), multialignment='center')

ax[1].set_xlabel('$x_0$')

ax[1].hist(theta_unknown_G[theta_unknown_S[:,0]!=-10][:,0], bins=bins, **plot_style_2, label='Gen.')
legend = ax[1].legend(
    loc='best',
    frameon=True)


ax[1].hist(theta_unknown_S[theta_unknown_S[:,0]!=-10][:,0], bins=bins, **plot_style_2, label='Detector')
legend = ax[1].legend(
    loc='best',
    frameon=True)
plt.setp(legend.get_title(), multialignment='center')

fig.show()

bins = np.linspace(-3,6, 101)
#bins = np.linspace(0,2, 101)
#bins = np.linspace(0,1,100)
fig, ax = plt.subplots(1,
                       2,
                       figsize=(12, 5),
                       constrained_layout=True,
                       sharey=True)

ax[0].set_xlabel('$x_1$')
ax[0].set_ylabel('Events per bin (normalized)')
ax[0].hist(theta0_G[theta0_G[:,0]!=-10][:,-1], bins=bins, **plot_style_2, label='Generation')
ax[0].hist(theta0_S[theta0_S[:,0]!=-10][:,-1], bins=bins, **plot_style_2, label='Simulation')
legend = ax[0].legend(
    loc='best',
    frameon=True)
plt.setp(legend.get_title(), multialignment='center')

ax[1].set_xlabel('$x_1$')

ax[1].hist(theta_unknown_G[theta_unknown_S[:,0]!=-10][:,-1], bins=bins, **plot_style_2, label='Gen.')
legend = ax[1].legend(
    loc='best',
    frameon=True)

ax[1].hist(theta_unknown_S[theta_unknown_S[:,0]!=-10][:,-1], bins=bins, **plot_style_2, label='Detector')
legend = ax[1].legend(
    loc='best',
    frameon=True)






plt.setp(legend.get_title(), multialignment='center')

fig.show()

## Run unfolding on data

In [ ]:
# import tensorflow as tf
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)

In [ ]:
%%time 
iterations =3# use ~4 iterations
num_observables= 6
K.clear_session()
weights, _, history = multifold(num_observables=num_observables,
                       iterations=iterations,
                       theta0_G=theta0_G,
                       theta0_S=theta0_S,
                       theta_unknown_S= theta_unknown_S,verbose=1)

In [ ]:
#list all data in history
# summarize history for accuracy
for i in range(iterations):
    fig,ax = plt.subplots(1,2,figsize=(16,6))
    ax[0].set_title('model loss')
    ax[0].set_xlabel('epoch number')
    ax[0].plot(history['step1'][i].history['loss'][1:],'o',label='Step 1 train')
    ax[0].plot(history['step1'][i].history['val_loss'][1:],'o',label='Step 1 test')

    ax[1].set_title('model accuracy')
    ax[1].plot(history['step1'][i].history['accuracy'][1:],'o',label='Step 1 train')
    ax[1].plot(history['step1'][i].history['val_accuracy'][1:],'o',label='Step 1 test')
    ax[1].legend()
    ax[1].set_xlabel('epoch number')
    ax[0].legend()

In [ ]:
for i in range(iterations):
    fig,ax = plt.subplots(1,2,figsize=(16,6))
    ax[0].set_title('model loss')
    ax[0].set_xlabel('epoch number')
    ax[0].plot(history['step2'][i].history['loss'][1:],'o',label='Step 2 train')
    ax[0].plot(history['step2'][i].history['val_loss'][1:],'o',label='Step 2 test')

    ax[1].set_title('model accuracy')
    ax[1].plot(history['step2'][i].history['accuracy'][1:],'o',label='Step 2 train')
    ax[1].plot(history['step2'][i].history['val_accuracy'][1:],'o',label='Step 2 test')
    ax[1].legend()
    ax[1].set_xlabel('epoch number')

    ax[0].legend()

## Incorporate weights into the dataframe

In [ ]:
mc_cut['weight_reco'] =  weights[-1, 0, :]
mc_cut['weight_truth'] = weights[-1, 1, :]

## Check weights

In [ ]:
plt.hist(np.log10(mc_cut['weight_truth']),range=(-10,10),bins=100,**plot_style_1,label='Weight truth')
plt.hist(np.log10(mc_cut['weight_reco']),range=(-10,10),bins=100,**plot_style_1, label='Weight reco')

plt.legend()
plt.xlabel('Log 10 weight truth')
plt.yscale('log')
plt.show()

#for i in range(len(weights)):
    #print ('Iteration %i'%i)
    #print(max(weights[i, 1, :]), min(weights[i, 1, :]))    
    #print(max(weights[i, 0, :]), min(weights[i, 0, :]))
    #print(' ')

In [ ]:
print(len(mc_cut))

mc_cut = mc_cut.query('weight_reco<1e4 and weight_reco>1e-3')
mc_cut = mc_cut.query('weight_truth<1e4 and weight_truth>1e-3')
print(len(mc_cut))

In [ ]:
fig, ax = plt.subplots(1,
                           3,
                           figsize=(15, 5),
                           constrained_layout=True)
ax = ax.flatten()
for i in range(len(weights)):

    print("ITERATION: {}".format(i + 1))
    #bins = np.linspace(5, 50, 45)
    
    
    ax[i].hist(np.log10(weights[i, 1, :][~np.isnan(weights[i, 1, :])]),bins=100,**plot_style_2,label='wgt truth')
    ax[i].hist(np.log10(weights[i, 0, :][~np.isnan(weights[i, 0, :])]),bins=100,**plot_style_2, label='wgt reco') 
    ax[i].legend()
    #ax[i].set_yscale('log')
    ax[i].set_title('iteration %i'%i)

    #ax[i].xlabel('Log 10 weight truth')

## Incorporate weights into the dataframe

In [ ]:
mc_cut.keys()

## MC at reco level reweighted vs data

In [ ]:
mc_name = 'MC PS'
altmc_name = 'MC'

In [ ]:
query = 'pass_reco==1 '
print('The query is:' , query)
label1 = mc_name +' reco'
label2 = mc_name +' wgt.'
label3 = 'Data reco'

color1='blueviolet'
color2='limegreen'
#color1='red'
#color2='blue'
for i in range(iterations):
    #if(i<2): continue

    print("ITERATION: {}".format(i + 1))
    bins = np.linspace(2, 5,100 )
    fig, ax = plt.subplots(3,
                           3,
                           figsize=(20, 20),
                           constrained_layout=True)
    bins = np.linspace(0.0, 1.0,40 )
    
    ax = ax.flatten()


    bins = np.linspace(-2, 2,20 )

    ax[0].set_xlabel(r'proton px',fontsize=22)
    ax[0].set_ylabel('Events per bin (Normalized)')
    ax[0].hist(mc_cut.query(query)['rec_px'],bins=bins, label=label1, **plot_style_2,color=color1)
    ax[0].hist(mc_cut.query(query)['rec_px'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label=label2,
               **plot_style_1)

    ax[0].hist(data.query(query)['rec_px'],
               bins=bins,
               label=label3,
               **plot_style_2,color=color2)
    
    ax[0].legend(frameon=False,loc='best')

    
    bins = np.linspace(-2, 2,20 )

    ax[1].set_xlabel(r'proton py',fontsize=22)
    ax[1].set_ylabel('Events per bin (Normalized)')
    ax[1].hist(mc_cut.query(query)['rec_py'],bins=bins, label=label1, **plot_style_2,color=color1)
    ax[1].hist(mc_cut.query(query)['rec_py'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label=label2,
               **plot_style_1)

    ax[1].hist(data.query(query)['rec_py'],
               bins=bins,
               label=label3,
               **plot_style_2,color=color2)
    
    ax[1].legend(frameon=False,loc='best')

    
    bins = np.linspace(-2, 5,20 )

    ax[2].set_xlabel(r'proton pz',fontsize=22)
    ax[2].set_ylabel('Events per bin (Normalized)')
    ax[2].hist(mc_cut.query(query)['rec_pz'],bins=bins, label=label1, **plot_style_2,color=color1)
    ax[2].hist(mc_cut.query(query)['rec_pz'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label=label2,
               **plot_style_1)

    ax[2].hist(data.query(query)['rec_pz'],
               bins=bins,
               label=label3,
               **plot_style_2,color=color2)
    
    ax[2].legend(frameon=False,loc='best')
    

    ax[3].set_xlabel(r'pip px',fontsize=22)
    ax[3].set_ylabel('Events per bin (Normalized)')
    ax[3].hist(mc_cut.query(query)['rec_pipx'],bins=bins, label=label1, **plot_style_2,color=color1)
    ax[3].hist(mc_cut.query(query)['rec_pipx'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label=label2,
               **plot_style_1)

    ax[3].hist(data.query(query)['rec_pipx'],
               bins=bins,
               label=label3,
               **plot_style_2,color=color2)
    
    ax[3].legend(frameon=False,loc='best')

    
    bins = np.linspace(-2, 2,20 )

    ax[4].set_xlabel(r' pip py',fontsize=22)
    ax[4].set_ylabel('Events per bin (Normalized)')
    ax[4].hist(mc_cut.query(query)['rec_pipy'],bins=bins, label=label1, **plot_style_2,color=color1)
    ax[4].hist(mc_cut.query(query)['rec_pipy'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label=label2,
               **plot_style_1)

    ax[4].hist(data.query(query)['rec_pipy'],
               bins=bins,
               label=label3,
               **plot_style_2,color=color2)
    
    ax[4].legend(frameon=False,loc='best')

    
    bins = np.linspace(-2, 5,20 )

    ax[5].set_xlabel(r'pip pz',fontsize=22)
    ax[5].set_ylabel('Events per bin (Normalized)')
    ax[5].hist(mc_cut.query(query)['rec_pipz'],bins=bins, label=label1, **plot_style_2,color=color1)
    ax[5].hist(mc_cut.query(query)['rec_pipz'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label=label2,
               **plot_style_1)

    ax[5].hist(data.query(query)['rec_pipz'],
               bins=bins,
               label=label3,
               **plot_style_2,color=color2)
    
    ax[5].legend(frameon=False,loc='best')
    
    
    ax[6].set_xlabel(r'pim px',fontsize=22)
    ax[6].set_ylabel('Events per bin (Normalized)')
    ax[6].hist(mc_cut.query(query)['rec_pimx'],bins=bins, label=label1, **plot_style_2,color=color1)
    ax[6].hist(mc_cut.query(query)['rec_pimx'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label=label2,
               **plot_style_1)

    ax[6].hist(data.query(query)['rec_pimx'],
               bins=bins,
               label=label3,
               **plot_style_2,color=color2)
    
    ax[6].legend(frameon=False,loc='best')

    
    bins = np.linspace(-2, 2,20 )

    ax[7].set_xlabel(r' pim py',fontsize=22)
    ax[7].set_ylabel('Events per bin (Normalized)')
    ax[7].hist(mc_cut.query(query)['rec_pimy'],bins=bins, label=label1, **plot_style_2,color=color1)
    ax[7].hist(mc_cut.query(query)['rec_pimy'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label=label2,
               **plot_style_1)

    ax[7].hist(data.query(query)['rec_pimy'],
               bins=bins,
               label=label3,
               **plot_style_2,color=color2)
    
    ax[7].legend(frameon=False,loc='best')

    
    bins = np.linspace(-2, 5,20 )

    ax[8].set_xlabel(r'pim pz',fontsize=22)
    ax[8].set_ylabel('Events per bin (Normalized)')
    ax[8].hist(mc_cut.query(query)['rec_pimz'],bins=bins, label=label1, **plot_style_2,color=color1)
    ax[8].hist(mc_cut.query(query)['rec_pimz'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label=label2,
               **plot_style_1)

    ax[8].hist(data.query(query)['rec_pimz'],
               bins=bins,
               label=label3,
               **plot_style_2,color=color2)
    
    ax[8].legend(frameon=False,loc='best')
    
    
    
    plt.tight_layout()
    #fig.savefig('figures/RealDataMC_comparison_inclusivejet_%i_iter_%s_%s_%s_data_%s.png'%(i,mc_name,mc_tag,period,data_tag)) 
    
    

## Plot unfolded results

In [ ]:
plt.hist(mc_cut.query('pass_reco==0')['weight_truth'],bins=100,label='weights truth, not reco')
plt.hist(mc_cut.query('pass_reco==1')['weight_truth'],bins=100, label='weights truth, reco')
plt.legend()
plt.show()

In [ ]:

query = 'gen_pz>1.0'#and pass_reco==1'
print('The query is:' , query)

label1 = mc_name +' truth'
label2 = 'unfolded + acc. corr'
label3 = altmc_name + ' truth (target)'


for i in range(len(weights)):
    #if(i<2): continue
    print("ITERATION: {}".format(i + 1))
    bins = np.linspace(2, 5,100 )
    #bins = np.linspace(5, 50, 45)

    fig, ax = plt.subplots(3,
                           3,
                           figsize=(20, 20),
                           constrained_layout=True)
    
    ax = ax.flatten()
    
    bins = np.linspace(-1.0, 5.0,40 )

    ax[0].set_xlabel('proton pz',fontsize=22)
    ax[0].set_ylabel('Events per bin (Normalized)')
    ax[0].hist(mc_cut.query(query)['gen_pz'], bins=bins, label=label1, **plot_style_2)
    ax[0].hist(mc_cut.query(query)['gen_pz'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1)

    ax[0].hist(altmc.query(query)['gen_pz'],
               bins=bins,
               label=label3,
               **plot_style_2)
    
    ax[0].legend(frameon=False)
    
    bins = np.linspace(-2.5, 2.5,40 )

    ax[1].set_xlabel('proton px',fontsize=22)
    ax[1].set_ylabel('Events per bin (Normalized)')
    ax[1].hist(mc_cut.query(query)['gen_px'], bins=bins, label=label1, **plot_style_2)
    ax[1].hist(mc_cut.query(query)['gen_px'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1)

    ax[1].hist(altmc.query(query)['gen_px'],
               bins=bins,
               label=label3,
               **plot_style_2)
    
    ax[1].legend(frameon=False)
    bins = np.linspace(-2.5, 2.5,40 )

    ax[2].set_xlabel('proton py',fontsize=22)
    ax[2].set_ylabel('Events per bin (Normalized)')
    ax[2].hist(mc_cut.query(query)['gen_py'], bins=bins, label=label1, **plot_style_2)
    ax[2].hist(mc_cut.query(query)['gen_py'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1)

    ax[2].hist(altmc.query(query)['gen_py'],
               bins=bins,
               label=label3,
               **plot_style_2)
    
    ax[2].legend(frameon=False)

    
    
    bins = np.linspace(-1.0, 5.0,40 )

    ax[3].set_xlabel('pip pz',fontsize=22)
    ax[3].set_ylabel('Events per bin (Normalized)')
    ax[3].hist(mc_cut.query(query)['gen_pipz'], bins=bins, label=label1, **plot_style_2)
    ax[3].hist(mc_cut.query(query)['gen_pipz'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1)

    ax[3].hist(altmc.query(query)['gen_pipz'],
               bins=bins,
               label=label3,
               **plot_style_2)
    
    ax[3].legend(frameon=False)
    
    bins = np.linspace(-2.5, 2.5,40 )

    ax[4].set_xlabel('pip px',fontsize=22)
    ax[4].set_ylabel('Events per bin (Normalized)')
    ax[4].hist(mc_cut.query(query)['gen_pipx'], bins=bins, label=label1, **plot_style_2)
    ax[4].hist(mc_cut.query(query)['gen_pipx'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1)

    ax[4].hist(altmc.query(query)['gen_pipx'],
               bins=bins,
               label=label3,
               **plot_style_2)
    
    ax[4].legend(frameon=False)
    bins = np.linspace(-2.5, 2.5,40 )

    ax[5].set_xlabel('pip py',fontsize=22)
    ax[5].set_ylabel('Events per bin (Normalized)')
    ax[5].hist(mc_cut.query(query)['gen_pipy'], bins=bins, label=label1, **plot_style_2)
    ax[5].hist(mc_cut.query(query)['gen_pipy'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1)

    ax[5].hist(altmc.query(query)['gen_pipy'],
               bins=bins,
               label=label3,
               **plot_style_2)
    
    ax[5].legend(frameon=False)

    
    
    bins = np.linspace(-1.0, 5.0,40 )

    ax[6].set_xlabel('pim pz',fontsize=22)
    ax[6].set_ylabel('Events per bin (Normalized)')
    ax[6].hist(mc_cut.query(query)['gen_pimz'], bins=bins, label=label1, **plot_style_2)
    ax[6].hist(mc_cut.query(query)['gen_pimz'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1)

    ax[6].hist(altmc.query(query)['gen_pimz'],
               bins=bins,
               label=label3,
               **plot_style_2)
    
    ax[6].legend(frameon=False)
    
    bins = np.linspace(-2.5, 2.5,40 )

    ax[7].set_xlabel('pim px',fontsize=22)
    ax[7].set_ylabel('Events per bin (Normalized)')
    ax[7].hist(mc_cut.query(query)['gen_pimx'], bins=bins, label=label1, **plot_style_2)
    ax[7].hist(mc_cut.query(query)['gen_pimx'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1)

    ax[7].hist(altmc.query(query)['gen_pimx'],
               bins=bins,
               label=label3,
               **plot_style_2)
    
    ax[7].legend(frameon=False)
    bins = np.linspace(-2.5, 2.5,40 )

    ax[8].set_xlabel('pim py',fontsize=22)
    ax[8].set_ylabel('Events per bin (Normalized)')
    ax[8].hist(mc_cut.query(query)['gen_pimy'], bins=bins, label=label1, **plot_style_2)
    ax[8].hist(mc_cut.query(query)['gen_pimy'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1)

    ax[8].hist(altmc.query(query)['gen_pimy'],
               bins=bins,
               label=label3,
               **plot_style_2)
    
    ax[8].legend(frameon=False)
    

In [ ]:

query = 'gen_pz>.50'#and pass_reco==1'
print('The query is:' , query)

label1 = mc_name +' truth'
label2 = 'unfolded'
label3 = altmc_name + ' truth (target)'


for i in range(len(weights)):
    #if(i<2): continue
    print("ITERATION: {}".format(i + 1))
    bins = np.linspace(2, 5,100 )
    #bins = np.linspace(5, 50, 45)

    fig, ax = plt.subplots(1,
                           3,
                           figsize=(18, 6),
                           constrained_layout=True)
    
    ax = ax.flatten()
    
    bins = np.linspace(-1.0, 5.0,40 )

    ax[0].set_xlabel('proton pz',fontsize=22)
    ax[0].set_ylabel('Events per bin (Normalized)')
    ax[0].hist(mc_cut.query(query)['gen_pz'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1A)

    ax[0].hist(altmc.query(query)['gen_pz'],
               bins=bins,
               label=label3,
               **plot_style_2A)
    
    ax[0].hist(data.query(query)['gen_pz'],
               bins=bins,
               label=' "data "',
               **plot_style_2A)
    
    
    ax[0].legend(frameon=False)
    
    bins = np.linspace(-2.5, 2.5,40 )

    ax[1].set_xlabel('proton px',fontsize=22)
    ax[1].set_ylabel('Events per bin (Normalized)')
    ax[1].hist(mc_cut.query(query)['gen_px'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1A)

    ax[1].hist(altmc.query(query)['gen_px'],
               bins=bins,
               label=label3,
               **plot_style_2A)
    ax[1].hist(data.query(query)['gen_px'],
               bins=bins,
               label=' "data "',
               **plot_style_2A)
    
    ax[1].legend(frameon=False)
    bins = np.linspace(-2.5, 2.5,40 )

    ax[2].set_xlabel('proton py',fontsize=22)
    ax[2].set_ylabel('Events per bin (Normalized)')
    ax[2].hist(mc_cut.query(query)['gen_py'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label=label2,
               **plot_style_1A)

    ax[2].hist(altmc.query(query)['gen_py'],
               bins=bins,
               label=label3,
               **plot_style_2A)
    
    ax[2].legend(frameon=False)
    ax[2].hist(data.query(query)['gen_py'],
               bins=bins,
               label=' "data "',
               **plot_style_2A)
    
    
    

## Data/MC agreement with differential in mass

In [ ]:
#bins = np.logspace(np.log10(0.01),np.log10(3),8)
bins = np.linspace(0,2,50)


fig, axs = plt.subplots(1, 3,sharey=True, figsize=(24,6), gridspec_kw={'wspace':0, 'hspace':0})

for counter,edges in enumerate([(0.2,0.4), (0.4,0.6),(0.6,1.0)]):
    #cut = ' Q2 > %2.2f and Q2 <%2.2f and pass_reco==1'%(edges[0],edges[1])
    cut = 'rec_ppt>%2.2f and rec_ppt<%2.2f'%(edges[0],edges[1])
    print (cut)
    print ('###########')

    axs[counter].xaxis.set_label_text('mass', fontsize=25)


    
    axs[counter].hist(mc_cut.query(cut)['rec_mass'],bins=bins, label=mc_name+' reco', **plot_style_2,color=color1)
    axs[counter].hist(mc_cut.query(cut)['rec_mass'],
               bins=bins,
               weights=mc_cut.query(cut)['weight_reco'],
               label=mc_name +' reco wgt.',
               **plot_style_1)
    axs[counter].hist(data.query(cut)['rec_mass'],
               bins=bins,
               label='Data reco',
               **plot_style_2,color=color2)    
    
    
    axs[counter].legend(prop={'size': 15}, frameon=False, loc='upper left')
    axs[counter].set_title(' $ %2.2f < t < %2.2f$ GeV'%(edges[0],edges[1]))


    #axs[counter].text(1.5,2.9,  r'$\langle x \rangle = %2.2f, \langle Q^{2} \rangle = %2.0f \ \mathrm{GeV}^{2}$'%(data_cut.query(cut)['x'].mean(), data_cut.query(cut)['Q2'].mean()), fontsize=18)
    #axs[counter].set_yscale('log')
    #axs[counter].set_xscale('log')

 
axs[0].yaxis.set_label_text('Normalized entries', fontsize=25)

#fig.savefig('figures/RealDataMCcomparison_dphi_diff_%s_%s_%s_data_%s.png'%(mc_name,mc_tag,period,data_tag)) 


In [ ]:
print(mc_cut.query('pass_reco==0').shape[0])
print(mc_cut.query('pass_reco==1').shape[0])

print(altmc_cut.query('pass_reco==0').shape[0])
print(altmc_cut.query('pass_reco==1').shape[0])

In [ ]:
bins = np.linspace(0, 2.5,40 )
query = 'pass_reco==1'
plt.hist(mc_cut.query(query)['gen_ppt'], bins=bins, label='MC1', **plot_style_2A)
plt.hist(mc_cut.query(query)['gen_ppt'],
               bins=bins,
               weights=mc_cut.query(query)['weight_truth'],
               label='MC1 wtg',
               **plot_style_1A)

plt.hist(altmc.query(query)['gen_ppt'],
               bins=bins,
               label='MC2 truth',
               **plot_style_2A)
    
plt.legend(frameon=False)

In [ ]:
bins = np.linspace(0, 2.5,40 )
query = 'pass_reco==1'
plt.hist(mc_cut.query(query)['rec_mass'], bins=bins, label='MC1', **plot_style_2)
plt.hist(mc_cut.query(query)['rec_mass'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label='MC1 wtg',
               **plot_style_1)

plt.hist(altmc.query(query)['rec_mass'],
               bins=bins,
               label='MC2',
               **plot_style_2)
    
plt.legend(frameon=False)

In [ ]:
plt.hist(altmc_cut.query('pass_reco==1')['rec_MM'],bins=100,**plot_style_2)
plt.hist(mc_cut.query('pass_reco==1')['rec_MM'],bins=100,**plot_style_2)
#plt.hist(mc_cut.query('pass_reco==1')['gen_MM'],bins=100,**plot_style_2)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
query = 'pass_reco==1'
bins = np.linspace(0,4,30)
plt.hist(mc_cut.query(query)['rec_mass'],
               bins=bins,
               weights=mc_cut.query(query)['weight_reco'],
               label='Rapgap reco weighted', color='black', histtype='step', lw=2
               )

plt.hist(altmc_cut.query(query)['rec_mass'],
               bins=bins,
               label='Data reco',
               color=color2, alpha=0.5)
plt.legend()
#plt.yscale('log')

plt.xlabel('EGamma [GeV]',fontsize=22)
plt.ylabel('Events per bin ')
#plt.xscale('log')
plt.legend(frameon=False,loc='best')
#fig.savefig('figures/jetpt_reco.png'%i)  



In [ ]:
altmc_cut = altmc

## Define fiducial region of the measurement

## Plot data, and unfolded results and MC truth

In [ ]:
altmc_name = 'MC realistic'

In [ ]:
fig = plt.figure(figsize=(8,6))
cut = fiducialregion
bins = np.linspace(0,5,100)
cut = 'pass_reco==0'
plt.hist(mc_cut.query(cut)['gen_pz'],bins=bins,weights=mc_cut.query(cut)['weight_truth'],label = 'Unfolded',**plot_style_1)
plt.hist(mc_cut.query(cut)['gen_pz'], bins=bins,label = '%s truth'%(mc_name),**plot_style_2)
plt.hist(altmc.query(cut)['gen_pz'], bins=bins,label = '%s truth'%(altmc_name),**plot_style_2)


plt.legend()
plt.ylabel('Normalized entries', fontsize=22)



plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16, 6))

cut = ' pass_reco==1'

bins = np.linspace(0,2,30)
#bins = np.array([10,14,21,29,47,71,127])

ax[0].hist(mc_cut.query(cut)['gen_mass'],bins=bins,label= mc_name+' Truth',**plot_style_2)

num,x,_ = ax[0].hist(altmc_cut.query(cut)['gen_mass'],bins=bins,label=altmc_name+' Truth',**plot_style_2)
den,x,_ = ax[0].hist(mc_cut.query(cut)['gen_mass'],bins=bins,weights=mc_cut.query(cut)['weight_truth'],label = altmc_name+' unfolded',**plot_style_1)
x= (x[1:] + x[:-1])/2.0
ratio = np.true_divide(num,den)

ax[0].legend()
ax[0].set_xlabel('$p_{T}^{jet}$ [GeV]')
ax[1].set_xlabel('$p_{T}^{jet}$ [GeV]')

ax[0].set_ylabel('Events per bin (Normalized)')
ax[1].set_ylabel('Unfolded/Truth')


ax[0].set_title('$Q^{2}>150$ GeV$^{2}$, $0.2<y<0.7$, $p_{T}^{jet}>10$ GeV',fontsize=20)

ax[1].plot(x,ratio,'o',color='black')
ax[1].fill_between(x, 0.90,1.1,alpha=0.2,color='black')
ax[1].set_ylim([0.5,1.5])
print(ratio)
#lt.tight_layout()
#fig.savefig('figures/Closure_jetpt_%s_%s.png'%(mc_name, altmc_name))  

plt.show()

## Final Cross section measurements. Save them in pickle files

In [ ]:
label_DISselection_truth = r'$Q^{2}>150$ GeV$^{2}, 0.2<y<0.7$'

In [ ]:
lumi = 139 #pb-1. This is for the 0607 dataset

## Calculate total jet cross-section


In [ ]:
fiducialregion = 'gen_Q2>150 and gen_y>0.2 and gen_y<0.7 and genjet_pt>10 and genjet_eta<2.5 and genjet_eta>-1.0'
bins = np.logspace(np.log10(10),np.log10(100),7)
y,x = np.histogram(mc_cut.query(fiducialregion)['genjet_pt'],bins=bins,weights=mc_cut.query(fiducialregion)['weight_truth'])
total = np.sum(y)/lumi
print('Total cross-section is: %f' %total)

In [ ]:

fig = plt.figure(figsize=(8,6))
bins = np.logspace(np.log10(10),np.log10(100),7)

y,x = np.histogram(mc_cut.query(fiducialregion)['genjet_pt'],bins=bins,weights=mc_cut.query(fiducialregion)['weight_truth'])
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y = y/lumi
y = y/total
y = y/(xerr*2.0)

print(y)
print(x)
plt.errorbar(x,y, xerr=xerr, fmt='o',ls='none',label='data unfold')
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.xlabel(r'jet $p_{T}$ $[\mathrm{GeV}]$',fontsize=18)
plt.ylabel(r'1/$\sigma_{jet}$ d$\sigma/dp_{T}$ ',fontsize=18)

plt.yscale('log')
plt.xscale('log')

plt.title(label_DISselection_truth, fontsize=18)

fig.savefig('figures/CrossSection_jetpt_%s.png'%(mc_name)) 

with open('outputfiles/CrossSection_jetpt_%s_%s_%s_data_%s.npy'%(mc_name,mc_tag,period,data_tag) , 'wb') as f:
    np.save(f, y)
    np.save(f, x)
    np.save(f, xerr)



In [ ]:
fig = plt.figure(figsize=(8,6))
bins = np.linspace(-1,2.5,6)

y,x = np.histogram(mc_cut.query(fiducialregion)['genjet_eta'],bins=bins,weights=mc_cut.query(fiducialregion)['weight_truth'])
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y = y/lumi
y = y/total
y = y/(xerr*2.0)

print(y)
print(x)
plt.errorbar(x,y, xerr=xerr, fmt='o',ls='none',label='data unfold')
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.xlabel(r'jet $\eta$',fontsize=18)
plt.ylabel(r'd$\sigma/d\eta$ ',fontsize=18)
#plt.yscale('log')
#plt.xscale('log')

plt.title(label_DISselection_truth, fontsize=18)

fig.savefig('figures/CrossSection_jeteta_%s.png'%(mc_name)) 

with open('outputfiles/CrossSection_jeteta_%s_%s_%s_data_%s.npy'%(mc_name,mc_tag,period,data_tag) , 'wb') as f:
    np.save(f, y)
    np.save(f, x)
    np.save(f, xerr)


In [ ]:
fig = plt.figure(figsize=(8,6))
bins = np.logspace(np.log10(0.03),np.log10(3),9) -0.03
bins[0] = 0
y,x = np.histogram(mc_cut.query(fiducialregion)['genjet_qtnorm'],bins=bins,weights=mc_cut.query(fiducialregion)['weight_truth'])
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y = y/lumi
y = y/total
y = y/(xerr*2.0)

print(y)
print(x)

plt.errorbar(x,y, xerr=xerr,fmt='o',ls='none',label='data unfold')
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.xlabel(r'jet $q_{T}/Q$ ',fontsize=18)
plt.ylabel(r'd$\sigma/dq_{T}/Q $ ',fontsize=18)
plt.yscale('log')
plt.title(label_DISselection_truth, fontsize=18)
#plt.xscale('log')
fig.savefig('figures/CrossSection_jetqt_%s.png'%(mc_name)) 

with open('outputfiles/CrossSection_qt_%s_%s_%s_data_%s.npy'%(mc_name,mc_tag,period,data_tag) , 'wb') as f:
    np.save(f, y)
    np.save(f, x)
    np.save(f, xerr)

In [ ]:
fig = plt.figure(figsize=(8,6))

bins = np.linspace(np.pi/2.0,np.pi,10) 
y,x = np.histogram(mc_cut.query(fiducialregion)['genjet_dphi'],bins=bins,weights=mc_cut.query(fiducialregion)['weight_truth'])
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y = y/lumi
y = y/total
y = y/(xerr*2.0)

plt.errorbar(x,y, xerr=xerr,fmt='o',ls='none',label='data unfold')
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.xlabel(r'$\Delta\phi$ ',fontsize=18)
plt.ylabel(r'd$\sigma/d\Delta\phi  $ ',fontsize=18)
plt.yscale('log')
plt.title(label_DISselection_truth, fontsize=18)
fig.savefig('figures/CrossSection_jetdphi_%s.png'%(mc_name)) 

with open('outputfiles/CrossSection_dphi_%s_%s_%s_data_%s.npy'%(mc_name,mc_tag,period,data_tag) , 'wb') as f:
    np.save(f, y)
    np.save(f, x)
    np.save(f, xerr)
#plt.xscale('log')

In [ ]:
print('%s_%s_%s_data_%s.npy'%(mc_name,mc_tag,period,data_tag))

# FIN

## Checking stability of the result vs number of iterations

In [ ]:
bins = np.logspace(np.log10(10),np.log10(100),7)


y,x = np.histogram(mc_cut.query(fiducialregion)['genjet_pt'],bins=bins,weights=mc_cut.query(fiducialregion)['weight_truth'])
total = np.sum(y)/lumi
print('Total cross-section is: %2.12f' %total)


y,x = np.histogram(mc_cut.query(fiducialregion)['genjet_pt'],bins=bins,weights=mc_cut.query(fiducialregion)['weight_truth_next'])
total_next = np.sum(y)/lumi
print('Total cross-section is: %2.12f' %total_next)

y,x = np.histogram(mc_cut.query(fiducialregion)['genjet_pt'],bins=bins,weights=mc_cut.query(fiducialregion)['weight_truth_previous'])
total_previous = np.sum(y)/lumi
print('Total cross-section is: %2.12f' %total_previous)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16, 6))
bins = np.logspace(np.log10(10),np.log10(100),7)


cut = fiducialregion

ynom, x  = np.histogram(mc_cut.query(cut)['genjet_pt'],bins=bins,weights=mc_cut.query(cut)['weight_truth'])
ynext, x = np.histogram(mc_cut.query(cut)['genjet_pt'],bins=bins,weights=mc_cut.query(cut)['weight_truth_next'])
ydo, x   = np.histogram(mc_cut.query(cut)['genjet_pt'],bins=bins,weights=mc_cut.query(cut)['weight_truth_previous'])
xerr = (x[1:] - x[:-1])/2.0
ynom = ynom/total
ynext = ynext/total_next
ydo   = ydo/total_previous
ynom = ynom/xerr
ynext = ynext/xerr
ydo  = ydo/xerr
x = (x[1:]+x[:-1])/2
ax[0].errorbar(x,ynom,xerr=xerr,fmt='o',label='iterations=%i'%(iterations-1))
ax[0].errorbar(x,ynext,xerr=xerr,fmt='o',label='iterations=%i'%(iterations))
#ax[0].errorbar(x,ydo,xerr=xerr,fmt='o',label='iterations=%i'%(iterations-2))
ax[0].legend()
ax[0].set_yscale('log')


ratio = np.true_divide(ynext,ynom)
ax[1].plot(x,ratio,'o',label='iterations=%i'%(iterations))
#ratio = np.true_divide(ydo,ynom)
#ax[1].plot(x,ratio,'o',label='iterations=%i'%(iterations-2))


ax[1].legend()


ax[1].fill_between(x, 0.95,1.05,alpha=0.2,color='black')
ax[1].set_ylim([0.5,1.5])
ax[0].set_title('$Q^{2}>150$ GeV$^{2}$, $0.2<y<0.7$, $p_{T}^{jet}>10$ GeV',fontsize=20)

ax[0].set_ylabel(r'1/$\sigma_{jet}$ d$\sigma/dp_{T}$ ',fontsize=18)
ax[1].set_ylabel(' Ratio to nominal')
ax[1].set_xlabel(r'jet $p_{T}$ $[\mathrm{GeV}]$',fontsize=18)
ax[0].set_xlabel(r'jet $p_{T}$ $[\mathrm{GeV}]$',fontsize=18)

#ax[0].set_ylim(bottom=1)
fig.savefig('figures/Iterations_jetpt_%s_%s_%s_data_%s.png'%(mc_name,mc_tag,period,data_tag)) 

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16, 6))
bins = np.linspace(np.pi/2.0,np.pi,20) 

cut = fiducialregion

ynom, x  = np.histogram(mc_cut.query(cut)['genjet_dphi'],bins=bins,weights=mc_cut.query(cut)['weight_truth'])
ynext, x = np.histogram(mc_cut.query(cut)['genjet_dphi'],bins=bins,weights=mc_cut.query(cut)['weight_truth_next'])
ydo, x   = np.histogram(mc_cut.query(cut)['genjet_dphi'],bins=bins,weights=mc_cut.query(cut)['weight_truth_previous'])
xerr = (x[1:] - x[:-1])/2.0
ynom = ynom/total
ynext = ynext/total_next
ydo   = ydo/total_previous
ynom = ynom/xerr
ynext = ynext/xerr
ydo  = ydo/xerr
x = (x[1:]+x[:-1])/2
ax[0].errorbar(x,ynom,xerr=xerr,fmt='o',label='iterations=%i'%(iterations-1))
ax[0].errorbar(x,ynext,xerr=xerr,fmt='o',label='iterations=%i'%(iterations))
#ax[0].errorbar(x,ydo,xerr=xerr,fmt='o',label='iterations=%i'%(iterations-2))
ax[0].legend()
ax[0].set_yscale('log')


ratio = np.true_divide(ynext,ynom)
ax[1].plot(x,ratio,'o',label='iterations=%i'%(iterations))
#ratio = np.true_divide(ydo,ynom)
#ax[1].plot(x,ratio,'o',label='iterations=%i'%(iterations-2))


ax[1].legend()


ax[1].fill_between(x, 0.95,1.05,alpha=0.2,color='black')
ax[1].set_ylim([0.5,1.5])
ax[0].set_title('$Q^{2}>150$ GeV$^{2}$, $0.2<y<0.7$, $p_{T}^{jet}>10$ GeV',fontsize=20)

ax[0].set_ylabel(r'1/$\sigma_{jet}$ d$\sigma/dp_{T}$ ',fontsize=18)
ax[1].set_ylabel(' Ratio to nominal')



ax[1].set_xlabel(r'$\Delta\phi$ ',fontsize=18)
ax[0].set_xlabel(r'$\Delta\phi$ ',fontsize=18)

fig.savefig('figures/Iterations_jetdphi_%s_%s_%s_data_%s.png'%(mc_name,mc_tag,period,data_tag)) 

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16, 6))
bins = np.logspace(np.log10(0.03),np.log10(3.03),9) - 0.03
bins[0] = 0.0
cut = fiducialregion

ynom, x  = np.histogram(mc_cut.query(cut)['genjet_qtnorm'],bins=bins,weights=mc_cut.query(cut)['weight_truth'])
ynext, x = np.histogram(mc_cut.query(cut)['genjet_qtnorm'],bins=bins,weights=mc_cut.query(cut)['weight_truth_next'])
ydo, x   = np.histogram(mc_cut.query(cut)['genjet_qtnorm'],bins=bins,weights=mc_cut.query(cut)['weight_truth_previous'])
xerr = (x[1:] - x[:-1])/2.0
ynom = ynom/total
ynext = ynext/total_next
ydo   = ydo/total_previous
ynom = ynom/xerr
ynext = ynext/xerr
ydo  = ydo/xerr
x = (x[1:]+x[:-1])/2
ax[0].errorbar(x,ynom,xerr=xerr,fmt='o',label='iterations=%i'%(iterations-1))
ax[0].errorbar(x,ynext,xerr=xerr,fmt='o',label='iterations=%i'%(iterations))
#ax[0].errorbar(x,ydo,xerr=xerr,fmt='o',label='iterations=%i'%(iterations-2))
ax[0].legend()
ax[0].set_yscale('log')


ratio = np.true_divide(ynext,ynom)
ax[1].plot(x,ratio,'o',label='iterations=%i'%(iterations))
#ratio = np.true_divide(ydo,ynom)
#ax[1].plot(x,ratio,'o',label='iterations=%i'%(iterations-2))


ax[1].legend()


ax[1].fill_between(x, 0.95,1.05,alpha=0.2,color='black')
ax[1].set_ylim([0.5,1.5])
ax[0].set_title('$Q^{2}>150$ GeV$^{2}$, $0.2<y<0.7$, $p_{T}^{jet}>10$ GeV',fontsize=20)

ax[0].set_ylabel(r'1/$\sigma_{jet}$ d$\sigma/dq_{T}/Q$ ',fontsize=18)

ax[1].set_ylabel(' Ratio to nominal')
ax[0].set_xlabel(r'jet $q_{T}/Q$ ',fontsize=18)
ax[1].set_xlabel(r'jet $q_{T}/Q$ ',fontsize=18)

ax[0].legend()
#ax[0].set_xscale('log')
#ax[1].set_xscale('log')

fig.savefig('figures/Iterations_jetqt_%s_%s_%s_data_%s.png'%(mc_name,mc_tag,period,data_tag)) 

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16, 6))
bins = np.linspace(-1,2.5,8)
cut = fiducialregion

ynom, x  = np.histogram(mc_cut.query(cut)['genjet_eta'],bins=bins,weights=mc_cut.query(cut)['weight_truth'])
ynext, x = np.histogram(mc_cut.query(cut)['genjet_eta'],bins=bins,weights=mc_cut.query(cut)['weight_truth_next'])
ydo, x   = np.histogram(mc_cut.query(cut)['genjet_eta'],bins=bins,weights=mc_cut.query(cut)['weight_truth_previous'])
xerr = (x[1:] - x[:-1])/2.0
ynom = ynom/total
ynext = ynext/total_next
ydo   = ydo/total_previous
ynom = ynom/xerr
ynext = ynext/xerr
ydo  = ydo/xerr
x = (x[1:]+x[:-1])/2
ax[0].errorbar(x,ynom,xerr=xerr,fmt='o',label='iterations=%i'%(iterations-1))
ax[0].errorbar(x,ynext,xerr=xerr,fmt='o',label='iterations=%i'%(iterations))
#ax[0].errorbar(x,ydo,xerr=xerr,fmt='o',label='iterations=%i'%(iterations-2))
ax[0].legend()
#ax[0].set_yscale('log')


ratio = np.true_divide(ynext,ynom)
ax[1].plot(x,ratio,'o',label='iterations=%i'%(iterations))
#ratio = np.true_divide(ydo,ynom)
#ax[1].plot(x,ratio,'o',label='iterations=%i'%(iterations-2))


ax[1].legend()


ax[1].fill_between(x, 0.95,1.05,alpha=0.2,color='black')
ax[1].set_ylim([0.5,1.5])
ax[0].set_title('$Q^{2}>150$ GeV$^{2}$, $0.2<y<0.7$, $p_{T}^{jet}>10$ GeV',fontsize=20)

ax[0].set_ylabel(r'1/$\sigma_{jet}$ d$\sigma/d\eta$ ',fontsize=18)

ax[1].set_ylabel(' Ratio to nominal')

ax[1].set_xlabel(r'jet $\eta$',fontsize=18)
ax[0].set_xlabel(r'jet $\eta$',fontsize=18)
ax[0].set_ylim(bottom=0.0)

fig.savefig('figures/Iterations_jeteta_%s_%s_%s_data_%s.png'%(mc_name,mc_tag,period,data_tag)) 

plt.show()

## Termino.

## Compare to ZEUS data

In [ ]:

fig = plt.figure(figsize=(8,6))

cut = ' gen_Q2 > 150 and gen_y>0.2 and gen_y<0.7'
cut = cut + ' and genjet_pt>10'
bins = np.array([10,14,21,29,47,71,127])
y,x = np.histogram(mc_cut.query(cut)['genjet_pt'],bins=bins,weights=mc_cut.query(cut)['weight_truth'])
xerr = (x[1:] - x[:-1])/2.0
x = (x[1:]+x[:-1])/2
y = y/(xerr*2.0)
y = y/lumi


plt.errorbar(x,y, fmt='.',ls='none',label='data unfold')
plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.xlabel(r'$p_{T}$\ $[\mathrm{GeV}]$',fontsize=18)
plt.ylabel(r'd$\sigma$/$p_{T}$ [normalized]',fontsize=18)



#heray = np.array([2685, 1408,599.9,165.55,40.59,7.90, 0.873, 0.0433])
#heraxdo = np.array([6,8,10,14,21,29,47,71])
#heraxup = np.array([8,10,14,21,29,47,71,127])


heray = np.array([599.9,165.55,40.59,7.90, 0.873, 0.0433])
heraxdo = np.array([10,14,21,29,47,71])
heraxup = np.array([14,21,29,47,71,127])
heraerrx = (heraxup-heraxdo)/2.0
herax = (heraxup+heraxdo)/2.0

ratio = np.true_divide(heray,y)


print(len(herax))
print(len(heray))
plt.errorbar(herax, 0.139*heray,xerr=heraerrx,label='HERA')
plt.legend(loc='best',frameon=False,fontsize=18)


plt.yscale('log')
plt.xscale('log')

print(ratio)

# FINISH data unfolding. Now starting CLOSURE TEST

In [ ]:
K.clear_session()

## Bootstrapping

In [ ]:
print(len(data_cut))
print(len(mc_cut.query('pass_reco==1'))) ##only what is really reconstructed counts
print(len(mc_cut)) ##only what is really reconstructed counts


In [ ]:
epx_0_G       = mc_cut['gene_px']
epx_0_S       = mc_cut['e_px']

epy_0_G       = mc_cut['gene_py']
epy_0_S       = mc_cut['e_py']

epz_0_G       = mc_cut['gene_pz']
epz_0_S       = mc_cut['e_pz']

jetpt_0_G       = mc_cut['genjet_pt']
jetpt_0_S       = mc_cut['jet_pt']


jetphi_0_G       = mc_cut['genjet_phi']
jetphi_0_S       = mc_cut['jet_phi']


jeteta_0_G       = mc_cut['genjet_eta']
jeteta_0_S       = mc_cut['jet_eta']

jetdphi_0_G       = mc_cut['genjet_dphi']
jetdphi_0_S       = mc_cut['jet_dphi']


jetqt_0_G       = mc_cut['genjet_qt']
jetqt_0_S       = mc_cut['jet_qt']

weight_MC = mc_cut['wgt']


In [ ]:
theta0_G=np.c_[(epx_0_G, epy_0_G, epz_0_G,  jetpt_0_G, jeteta_0_G, jetphi_0_G, jetdphi_0_G, jetqt_0_G)]
theta0_S=np.c_[(epx_0_S, epy_0_S, epz_0_S,  jetpt_0_S, jeteta_0_S, jetphi_0_S, jetdphi_0_S, jetqt_0_S)]
 

In [ ]:
pass_reco = np.array(mc_cut['pass_reco'])
pass_truth = np.array(mc_cut['pass_truth'])
theta0_S[:,0][pass_reco==0] = -10
theta0_G[:,0][pass_truth==0] = -10

In [ ]:
%%time
bootstraps = 50
iterations =  5
nvariables = 8
weights = [] 


import timeit

#bootstrap_data = data_cut.sample(np.random.poisson(data_cut.shape[0],1)[0], replace=True)

for i in range(bootstraps):
    start = timeit.timeit()

    print ('%i th jackknifedata'%i)
    ## define the data input for the unfolding
    #bootstrap_data = data_cut.sample(data_cut.shape[0], replace=True)
    #add Poisson variation of total number of data events used:
    bootstrap_data = data_cut.sample(np.random.poisson(data_cut.shape[0],1)[0], replace=True)
    print(len(bootstrap_data), ' entries')
    epx_unknown_S       = bootstrap_data['e_px']
    epy_unknown_S       = bootstrap_data['e_py']
    epz_unknown_S       = bootstrap_data['e_pz'] 
    jetpt_unknown_S     = bootstrap_data['jet_pt']
    jeteta_unknown_S    = bootstrap_data['jet_eta']
    jetphi_unknown_S    = bootstrap_data['jet_phi']
    jetdphi_unknown_S   = bootstrap_data['jet_dphi']
    jetqt_unknown_S     = bootstrap_data['jet_qt']
    theta_unknown_S =np.c_[(epx_unknown_S, epy_unknown_S, epz_unknown_S, jetpt_unknown_S, 
                            jeteta_unknown_S, jetphi_unknown_S, jetdphi_unknown_S, jetqt_unknown_S)]

    
    K.clear_session()
    
    num_observables = 8
    i_weight, _ = multifold(num_observables=num_observables,
                       iterations=iterations,
                       theta0_G=theta0_G,
                       theta0_S=theta0_S,
                       theta_unknown_S= theta_unknown_S,
                       weights_MC_sim = weight_MC,
                       verbose =0
                      )
    
    
    
    weights.append(i_weight)
    #end = timeit.timeit()
    #print(end - start)


In [ ]:
print(len(weights))

## for k in range(bootstraps):
    for i in range(len(weights[k])):
        print (weights[k].shape)
        print((weights[k][i, 1, :]).shape)
        print((weights[k][i, 1, :]).shape)
        print((weights[k][i, 1:2, :][0]).shape)


In [ ]:
bins = 3.0*np.pi/2.0 + np.flip(np.geomspace(-np.pi/2.0, -np.pi,num=10))
print(bins)
bins = np.array([
1.57079633, 2.21890524 ,2.40373544 ,2.57486505,
 2.73330962, 2.88000945, 3.01583511, 3.14159265])

In [ ]:
bins_1 = np.linspace(np.pi/2.0,5*np.pi/6,3) 
bins_2 = np.linspace(5*np.pi/6,np.pi,4) 
bins = np.append(bins_1[:-1],bins_2)
print(bins)

In [ ]:
np.geomspace(1,4,7)-1

In [ ]:
bins = np.logspace(np.log10(0.03),np.log10(3.03),7) - 0.03
bins[0] = 0.0
print(bins)

In [ ]:
bins = np.logspace(np.log10(0.03),np.log10(3),7)
print(bins)

In [ ]:
#variable = 'eta'
#variable = 'pt'
#variable= 'dphi'
variable = 'qtnorm'

if 'eta' in variable:
    varname = 'genjet_eta'
    xlabel = 'jet $\eta$'
    ylabel = '$1/\sigma d\sigma/d\eta$'
    bins = np.linspace(-1,2.5,6) #jet eta
elif 'pt' in variable:
    varname = 'genjet_pt'
    xlabel = 'jet $p_{T}$ '
    bins = np.geomspace(10,100,7)
    ylabel = '$1/\sigma d\sigma/dp_{T}$'

elif 'dphi' in variable:
    varname = 'genjet_dphi'
    xlabel = '$\Delta\phi$'
    bins_1 = np.linspace(np.pi/2.0,5*np.pi/6,3) 
    bins_2 = np.linspace(5*np.pi/6,np.pi,6) 
    bins = np.append(bins_1[:-1],bins_2)
    ylabel = '$1/\sigma d\sigma/d\Delta\phi$'

    
elif 'qt' in variable:
    bins = np.logspace(np.log10(0.03),np.log10(3.03),9) - 0.03
    bins[0] = 0.0
    varname = 'genjet_qtnorm'
    xlabel = '$q_{T}/Q$'
    ylabel = '$1/\sigma d\sigma/dq_{T}/Q$'


In [ ]:
covariance_matrices = []
for it in range(5):
    myns = []
    fiducialregion = 'gen_Q2>150 and gen_y>0.2 and gen_y<0.7 and genjet_pt>10.0 and genjet_pt<100 and genjet_eta<2.5 and genjet_eta>-1.0'
    print ('Iteration %i'%it)

    for k in range(len(weights)):
    #for k in range(10):
   
        print ('k= %i'%k)
        mc_cut['weight_temp'] =weights[k][it, 1, :]
        cut = fiducialregion + 'and  weight_temp>1e-21 and weight_temp<1e21'
        y,x = np.histogram(mc_cut.query(cut)['genjet_pt'],bins=np.logspace(np.log10(10),np.log10(100),6),weights=mc_cut.query(cut)['weight_temp'])
        total = np.sum(y)
        #print('Total cross-section is: %f' %total)
        n,x  = np.histogram(mc_cut.query(cut)[varname],bins=bins,weights=mc_cut.query(cut)['weight_temp'])

        if(total>0):
            n = np.true_divide(n,total)
            myns+=[n]
            print('', n)
        pass
    xerr = (x[1:] - x[:-1])/2.0
    x = (x[1:]+x[:-1])/2
    myns = np.array(myns)
    cov = np.array([[np.mean((myns[:,j] - np.mean(myns[:,j]))*(myns[:,i] - np.mean(myns[:,i]))) for i in range(len(myns[0]))] for j in range(len(myns[0]))])
    corr = np.array([[cov[i,j]/(np.sqrt(cov[i,i])*np.sqrt(cov[j,j])) for i in range(len(myns[0]))] for j in range(len(myns[0]))])
    covariance_matrices.append(corr)


#print (len(myns))
#print (len(myns[0]))

In [ ]:

print(x)
print(xerr)
print(len(myns[0]))


ymean = [np.mean(myns[:,i]) for i in range(len(myns[0]))] 
yerr = [ np.sqrt(cov[i,i])/np.sqrt(len(myns[0]))  for i in range(len(myns[0]))] 
print (ymean)
print(yerr)


In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,6)) 

for i in range(10):
    ax[0].errorbar(x, myns[i]/(2.0*xerr),xerr=xerr, alpha=0.3)
ax[0].errorbar(x,ymean/(2.0*xerr), xerr=xerr,yerr=yerr,fmt='o',label='Average',color='k')
ax[0].legend()

ax[1].plot(x,100.0*np.true_divide(yerr,ymean),'o')
ax[1].set_title('Statistical error')
ax[0].set_title('Bootstrap results')

if 'pt' in variable:
    ax[0].set_yscale('log')

if 'dphi' in variable:
    ax[0].set_yscale('log')

if 'qt' in variable:
    ax[0].set_yscale('log')
ax[0].set_xlabel(xlabel)
ax[1].set_xlabel(xlabel)
ax[0].set_ylabel(ylabel)
ax[1].set_ylabel('Relative error [%]')

ax[1].set_ylim(bottom=0.0)
fig.savefig('figures/Bootstrapping_variable%s_%s_%s_%s_data_%s.png'%(variable,mc_name,mc_tag,period,data_tag)) 

#ax[0].set_xscale('log')

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(12,12)) 
minimo =-.40
maximo = 1.0

ax = ax.flatten()

im = ax[0].imshow(covariance_matrices[0],cmap='RdBu', vmin=minimo, vmax=1.0,
                             interpolation='none')
im = ax[1].imshow(covariance_matrices[1],cmap='RdBu', vmin=minimo, vmax=1.0,
                             interpolation='none')
im = ax[2].imshow(covariance_matrices[2],cmap='RdBu', vmin=minimo, vmax=1.0,
                  interpolation='none')

im = ax[3].imshow(covariance_matrices[3],cmap='RdBu', vmin=minimo, vmax =1.0,
                  interpolation='none')
im = ax[4].imshow(covariance_matrices[4],cmap='RdBu', vmin=minimo, vmax =1.0,
                  interpolation='none')
#im = ax[6].imshow(covariance_matrices[5],cmap='RdBu', vmin=minimo, vmax=1.0,
#                  interpolation='none')


ax[0].set_title('iteration #1')
ax[0].set_xlabel('bin number')
ax[0].set_ylabel('bin number')

ax[1].set_title('iteration #2')
ax[1].set_xlabel('bin number')
ax[1].set_ylabel('bin number')

ax[2].set_title('iteration #3')
ax[2].set_xlabel('bin number')
ax[2].set_ylabel('bin number')

ax[3].set_title('iteration #4')
ax[3].set_xlabel('bin number')
ax[3].set_ylabel('bin number')

ax[4].set_title('iteration #5')
ax[4].set_xlabel('bin number')
ax[4].set_ylabel('bin number')

fig.savefig('figures/CorrelationMatrices_%s_%s_%s_data_%s.png'%(mc_name,mc_tag,period,data_tag)) 


plt.colorbar(im,ax=ax)
plt.show()

In [ ]:
from numpy.linalg import inv
x = [] 
rhos= []
for it in range(3): #iterations
    temp = []
    cov = np.matrix(covariance_matrices[it])
    inversecov = inv(cov)
    for i in range(cov.shape[0]):
         if(cov[i,i]*inversecov[i,i]!=0):
            temp.append( np.sqrt(1- 1.0/(cov[i,i]*inversecov[i,i])))
    rho_it = np.average(temp)
    rhos.append(rho_it)
    x.append(it+1)
    print (rho_it)
print (x)
print (rhos)

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.plot(x,rhos,'-o')
plt.xlabel('Number of Multifold iterations',fontsize=20)
plt.ylabel('Mean global-correlation coefficient',fontsize=20)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)

In [ ]:
cov = np.array([[np.mean((myns[:,j] - np.mean(myns[:,j]))*(myns[:,i] - np.mean(myns[:,i]))) for i in range(len(myns[0]))] for j in range(len(myns[0]))])

In [ ]:
corr = np.array([[cov[i,j]/(np.sqrt(cov[i,i])*np.sqrt(cov[j,j])) for i in range(len(myns[0]))] for j in range(len(myns[0]))])

In [ ]:
fig, ax = plt.subplots(figsize=(8,6)) 
im = ax.imshow(covariance_matrices[2],cmap='RdBu', vmin=-1.0, vmax=1.0,
                             interpolation='none')
fig.colorbar(im,ax=ax)
plt.xlabel('bin number')
plt.ylabel('bin number')
plt.title(xlabel)
plt.show()
fig.savefig('figures/FinalCorrelationMatrix_%s_%s_%s_%s_data_%s.png'%(variable,mc_name,mc_tag,period,data_tag)) 
print(covariance_matrices[2])

In [ ]:
print (myns)